In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import pandas as pd
import numpy as np
import time

rootPath = "/content/drive/MyDrive/dataset/veremi"
veremiPath = "/veremi"
t2Path = "/veremiWithT2"
csvt2Path = "/simulationscsv"

os.chdir(rootPath+t2Path+csvt2Path)

os.listdir()[-1:]

['veins_maat.uc1.14505249.180205_165725.csv']

In [4]:
os.chdir(rootPath+t2Path+csvt2Path)
simulations = pd.Series(os.listdir()).sort_values().reset_index(drop=True)
simulations[0:45]

0     veins_maat.uc1.14505201.180205_165350.csv
1     veins_maat.uc1.14505202.180205_165350.csv
2     veins_maat.uc1.14505203.180205_165350.csv
3     veins_maat.uc1.14505204.180205_165350.csv
4     veins_maat.uc1.14505205.180205_165350.csv
5     veins_maat.uc1.14505206.180205_165418.csv
6     veins_maat.uc1.14505208.180205_165418.csv
7     veins_maat.uc1.14505209.180205_165417.csv
8     veins_maat.uc1.14505210.180205_165417.csv
9     veins_maat.uc1.14505211.180205_165416.csv
10    veins_maat.uc1.14505212.180205_165444.csv
11    veins_maat.uc1.14505213.180205_165444.csv
12    veins_maat.uc1.14505214.180205_165445.csv
13    veins_maat.uc1.14505215.180205_165431.csv
14    veins_maat.uc1.14505216.180205_165431.csv
15    veins_maat.uc1.14505217.180205_165512.csv
16    veins_maat.uc1.14505220.180205_165459.csv
17    veins_maat.uc1.14505221.180205_165511.csv
18    veins_maat.uc1.14505222.180205_165459.csv
19    veins_maat.uc1.14505223.180205_165513.csv
20    veins_maat.uc1.14505224.180205_165

In [5]:
os.chdir(rootPath+veremiPath)
simulations = pd.Series(os.listdir()).sort_values().reset_index(drop=True)
simulations[42:45]

42    veins_maat.uc1.14505250.180205_165725
43    veins_maat.uc1.14505251.180205_165724
44    veins_maat.uc1.14505252.180205_165724
dtype: object

In [ ]:
import json
start_time = time.time()
mid = 'work/ul/ul_vertsys/ul_wqy57'
end = 'veins-maat/simulations/securecomm2018/results'
os.chdir(rootPath+veremiPath)
lsimulations = len(simulations[42:45])
for idsim, sims in enumerate(simulations[42:45]):
  # if idsim < 1:
    os.chdir(rootPath+veremiPath+'/'+sims+'/'+mid+'/'+sims+'/'+end)
    df = pd.Series(os.listdir()).sort_values().reset_index(drop=True)
    df = df.drop(index=df.loc[df.str.contains('.json') == False].index).reset_index(drop=True)
    
    dataGT = []
    with open(df[0]) as f:
      for line in f:
        try:
          dataGT.append(json.loads(line.strip()))
        except ValueError:
          pass

    dataGT = pd.DataFrame(dataGT)
    # display(dataGT)

    dataLog = []
    lfiles = len(df[1:])
    for iditem, item in enumerate(df[1:], start=1):
      print('\r', 'reading simulation '+str(idsim+1)+' of '+str(lsimulations)+' |',
                " in {:.2f}".format(time.time() - start_time)+' sec',
                sep='', end='', flush=True)
      with open(df[iditem]) as f:
        for line in f:
          try:
            row = json.loads(line.strip())
            # add a receiver line to form the pair of communication
            row["receiver"] = int(df[iditem].split("-")[2])
            dataLog.append(row)
          except ValueError:
            pass

    dataLog = pd.DataFrame(dataLog)
    # add the last know receiver's position 
    vehicles = pd.Series(dataLog['receiver'].unique().astype(int)).sort_values().reset_index(drop=True)
    size = len(vehicles)
    # iv = vehicle index
    for iv, v in enumerate(vehicles):
      # if iv <= 1:
        # for each BSM (type 3) do this
        for tp in dataLog.loc[(dataLog['type'] == 3) & (dataLog['receiver'].astype(int) == v)].index:
          print('\r', 'simulation '+str(idsim+1)+' of '+str(lsimulations)+' |',
                ' vehicle '+str(iv+1)+' of '+str(size),
                " in {:.2f}".format(time.time() - start_time)+' sec',
                sep='', end='', flush=True)
          poslist = dataLog['pos'].iloc[dataLog.loc[(dataLog['receiver'].astype(int) == v) & (dataLog['type'] == 2)][0:tp].last_valid_index()]
          spdlist = dataLog['spd'].iloc[dataLog.loc[(dataLog['receiver'].astype(int) == v) & (dataLog['type'] == 2)][0:tp].last_valid_index()]
          dataLog.at[tp, 'pxRcv'] = poslist[0]
          dataLog.at[tp, 'pyRcv'] = poslist[1]
          dataLog.at[tp, 'pzRcv'] = poslist[2]
          dataLog.at[tp, 'sxRcv'] = spdlist[0]
          dataLog.at[tp, 'syRcv'] = spdlist[1]
          dataLog.at[tp, 'szRcv'] = spdlist[2]

    # Drop “Type2” in JSON files
    dataLog = dataLog.drop(dataLog[dataLog.type == 2].index).reset_index(drop=True)
    # Merge with ground truth to add labels
    dataLog = dataLog.merge(dataGT[['messageID', 'attackerType']])
    # display(dataLog)
    os.chdir(rootPath+t2Path+csvt2Path)
    print('\r',
          'saving',
          sep='', end='', flush=True)
    dataLog.to_csv(sims+'.csv')
    print('\r',
          sims+' done in {:.2f}'.format(time.time() - start_time)+' sec\n',
          sep='', end='', flush=True)

veins_maat.uc1.14505250.180205_165725 done in 11040.38 sec
simulation 2 of 3 | vehicle 343 of 494 in 21187.17 sec